In [117]:
import numpy as np
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
import re
import spacy
from keras.models import Sequential
from sklearn.utils.class_weight import compute_class_weight
from keras.layers import Dense, Embedding, LSTM, Dropout, MaxPooling1D, Conv1D, Flatten
from spacy import displacy
import nltk
from keras.datasets import imdb
import tensorflow as tf
from keras.preprocessing import text
from nltk.parse.dependencygraph import DependencyGraph
from prettytable import PrettyTable
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
import imdbb

In [118]:
stemmer = SnowballStemmer('english')

In [119]:
class Tweet:
    def __init__(self, count, hate_speech_count, offensive_language_count, neither_count, classs, tweet):
        """
        - count (int): The total count of the tweet.
        - hate (int): The count of hate speech in the tweet.
        - offensive (int): The count of offensive language in the tweet.
        - neither (int): The count of content classified as neither hate speech nor offensive.
        - classs (str): The classification of the tweet.
        - tweet (str): The text content of the tweet.
        """
        self.count = count
        self.hate = hate_speech_count
        self.offensive = offensive_language_count
        self.neither = neither_count
        self.classs = classs
        self.tweet = tweet
        self.stems = []
        self.tokens = []
        self.code = []
        self.tags = []
        self.t_code = []
        self.presence_word = []

    def __str__(self):
        return f"{self.count} ; {self.hate} ; {self.offensive} ; {self.neither} ; {self.classs} ;; {self.tweet}"
    
    def peacefullness(self):
        return self.neither_count / self.count
    
    def offensiveness(self):
        return self.offensive / self.count
    
    def hateness(self):
        return self.hate / self.count

In [120]:
Tweets = []

with open('./archive/train.csv', 'r+') as file:
    previous_line = ''

    # Initialize a list to accumulate the modified content
    final_content_lines = []

    # Read and accumulate non-empty lines
    for line in file:
        stripped_line = line.strip()

        if stripped_line and stripped_line[0].isdigit():
            # If the current line is not empty and starts with an integer, accumulate it
            final_content_lines.append(stripped_line)
            previous_line = stripped_line
        else:
            # If the current line doesn't start with an integer, append it to the previous line
            previous_line += stripped_line

In [121]:
for line in final_content_lines:
    
    comma_indices = [index for index, char in enumerate(line) if char == ',']

    # Extracting substrings between commas
    count_str = line[0:comma_indices[0]].strip()
    hate_str = line[comma_indices[0]+1:comma_indices[1]].strip()
    offensive_str = line[comma_indices[1]+1:comma_indices[2]].strip()
    neither_str = line[comma_indices[2]+1:comma_indices[3]].strip()
    classs_str = line[comma_indices[3]+1:comma_indices[4]].strip()
    tweet_str = line[comma_indices[4]+1:].strip()

    # Converting to integers
    count = int(count_str) if count_str.isdigit() else None
    hate = int(hate_str) if hate_str.isdigit() else None
    offensive = int(offensive_str) if offensive_str.isdigit() else None
    neither = int(neither_str) if neither_str.isdigit() else None
    classs = int(classs_str) if classs_str.isdigit() else None

    # Creating an instance of the Tweet class
    tweet_instance = Tweet(count, hate, offensive, neither, classs_str, tweet_str)

    # Append the tweet instance to a list or do whatever you need to do with it
    Tweets.append(tweet_instance)

Set parameters

In [122]:
# (X_train, y_train), (X_test, y_test) = imdbb.load_imdb()

In [123]:
vocab_size = 200
maxlen = 200
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 100
epochs = 10

In [124]:
X_train = []
X_test  = []
y_train = []
y_test  = []

for t in Tweets:
    X_train.append(t.tweet)
    if t.hateness() > 0:
        y_train.append(1)
    else:
        y_train.append(0)


In [125]:
tokenizer = text.Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X_train)

In [126]:
X_train = tokenizer.texts_to_matrix(X_train)
X_test = tokenizer.texts_to_matrix(X_test)
y_train = np.array(y_train)

In [127]:
avg = 0 
max = 0
min = 0

for i in X_train:
    l = len(i)
    if l > max:
        max = l
    if l < min:
        min = l

    avg += l

avg /= len(X_train)

print(f"avg: {avg}, \tmin: {min}, \tmax: {max}, \tlen(X_train): {len(X_train)}")

count = 0
for i in y_train:
    if i == 1:
        count+=1


print("y positive percentage balance:", 100* count / len(y_train))

200.0 0 200 24783
y positive percentage balance: 20.1468748739055


In [128]:
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [129]:
model = Sequential()
model.add(Embedding(vocab_size,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.5))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu'))
model.add(MaxPooling1D())
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(hidden_dims, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [132]:
class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(y_train),
                                        y = y_train                                                    
                                    )
class_weights = dict(zip(np.unique(y_train), class_weights))

In [133]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1, class_weight=class_weights) # validation_data=(X_test, y_test)

Epoch 1/10
775/775 [==============================] - 84s 105ms/step - loss: 0.6693 - accuracy: 0.5988
Epoch 2/10
775/775 [==============================] - 79s 102ms/step - loss: 0.6273 - accuracy: 0.6860
Epoch 3/10
775/775 [==============================] - 80s 103ms/step - loss: 0.6132 - accuracy: 0.6985
Epoch 4/10
775/775 [==============================] - 73s 94ms/step - loss: 0.6011 - accuracy: 0.7097
Epoch 5/10
775/775 [==============================] - 62s 81ms/step - loss: 0.5868 - accuracy: 0.7163
Epoch 6/10
775/775 [==============================] - 65s 84ms/step - loss: 0.5778 - accuracy: 0.7228
Epoch 7/10
775/775 [==============================] - 66s 86ms/step - loss: 0.5664 - accuracy: 0.7317
Epoch 8/10
775/775 [==============================] - 67s 86ms/step - loss: 0.5577 - accuracy: 0.7389
Epoch 9/10
775/775 [==============================] - 68s 87ms/step - loss: 0.5497 - accuracy: 0.7419
Epoch 10/10
775/775 [==============================] - 69s 90ms/step - loss: 0.

In [134]:
model.save('models/hate-v3-CNN.h5')

/home/antoine/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
